<a href="https://colab.research.google.com/github/EvenSol/NeqSim-Colab/blob/master/notebooks/process/%20water_hammer_simulation_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Mathematical Foundation

### Governing Equations

Water hammer is described by the **continuity** and **momentum** equations:

**Continuity equation:**
$$\frac{\partial P}{\partial t} + \rho a^2 \frac{\partial v}{\partial x} = 0$$

**Momentum equation:**
$$\frac{\partial v}{\partial t} + \frac{1}{\rho} \frac{\partial P}{\partial x} + g \sin\theta + \frac{f v |v|}{2D} = 0$$

Where:
- $P$ = pressure (Pa)
- $v$ = velocity (m/s)
- $\rho$ = density (kg/m³)
- $a$ = wave speed (m/s)
- $f$ = friction factor
- $D$ = pipe diameter (m)
- $\theta$ = pipe inclination angle (rad)
- $g$ = gravitational acceleration (m/s²)

### Wave Speed (Joukowsky Formula)

The pressure wave propagates at speed $a$:

$$a = \sqrt{\frac{K/\rho}{1 + \frac{K D}{E t}}}$$

Where:
- $K$ = bulk modulus of fluid (Pa)
- $E$ = Young's modulus of pipe material (Pa)
- $t$ = pipe wall thickness (m)

For water in steel pipes: $a \approx 1000-1400$ m/s

### Joukowsky Pressure Rise

Instantaneous valve closure creates maximum pressure rise:

$$\Delta P_{max} = \rho a \Delta v$$

This is the theoretical maximum - friction and valve closure time reduce actual pressure rise.

### Method of Characteristics

The model uses the **Method of Characteristics (MOC)** to transform PDEs into ODEs:

**C+ characteristic:**
$$\frac{dP}{dt} + \rho a \frac{dv}{dt} = -\rho g a \sin\theta - \frac{\rho f a v |v|}{2D}$$

**C- characteristic:**
$$\frac{dP}{dt} - \rho a \frac{dv}{dt} = -\rho g a \sin\theta - \frac{\rho f a v |v|}{2D}$$

### Cavitation Detection

Cavitation occurs when pressure drops below vapor pressure:

$$P < P_{vapor}$$

The model automatically detects and handles cavitation zones.

## 2. Setup and Basic Usage

First, let's import NeqSim and create a liquid system.

In [ ]:
import neqsim
from neqsim.thermo import system, stream
from neqsim.process.equipment import pipeline
import numpy as np
import matplotlib.pyplot as plt

# Create a water system
fluid = system.SystemSrkEos(298.15, 10.0)  # 25°C, 10 bar
fluid.addComponent("water", 1.0)
fluid.setMixingRule("classic")
fluid.init(0)

print(f"Fluid: Water")
print(f"Temperature: {fluid.getTemperature():.2f} K ({fluid.getTemperature()-273.15:.1f}°C)")
print(f"Pressure: {fluid.getPressure():.2f} bar")
print(f"Density: {fluid.getPhase(0).getDensity():.2f} kg/m³")

## 3. Example 1: Classic Valve Closure

The classic water hammer scenario: instantaneous valve closure at the end of a horizontal pipe.

In [ ]:
# Create inlet stream with initial velocity
inlet = stream.Stream("inlet", fluid.clone())
inlet.setFlowRate(50.0, "kg/sec")  # 50 kg/s flow rate
inlet.setTemperature(298.15, "K")
inlet.setPressure(10.0, "bara")
inlet.run()

print(f"Inlet conditions:")
print(f"  Flow rate: {inlet.getFlowRate('kg/sec'):.2f} kg/s")
print(f"  Temperature: {inlet.getTemperature('K'):.2f} K")
print(f"  Pressure: {inlet.getPressure('bara'):.2f} bara")

In [ ]:
# Create water hammer pipe
wh_pipe = pipeline.WaterHammerPipe("ValveClosurePipe", inlet)
wh_pipe.setLength(1000.0)  # 1 km pipeline
wh_pipe.setDiameter(0.3)  # 300 mm diameter
wh_pipe.setRoughness(0.000046)  # Steel pipe roughness
wh_pipe.setWallThickness(0.01)  # 10 mm wall thickness
wh_pipe.setNumberOfSections(50)  # Spatial discretization

# Set initial conditions (steady flow)
initial_velocity = 2.0  # m/s
wh_pipe.setInitialVelocity(initial_velocity)

# Configure valve closure event
wh_pipe.setValveClosureTime(0.1)  # Close valve in 0.1 seconds
wh_pipe.setSimulationTime(5.0)  # Simulate 5 seconds

print(f"\nPipeline configuration:")
print(f"  Length: {wh_pipe.getLength():.0f} m")
print(f"  Diameter: {wh_pipe.getDiameter()*1000:.0f} mm")
print(f"  Initial velocity: {initial_velocity:.2f} m/s")
print(f"  Valve closure time: {wh_pipe.getValveClosureTime():.2f} s")

# Calculate theoretical Joukowsky pressure rise
wave_speed = 1200.0  # Approximate for water in steel
density = fluid.getPhase(0).getDensity()
joukowsky_dp = density * wave_speed * initial_velocity / 1e5  # Convert to bar
print(f"\nTheoretical Joukowsky pressure rise: {joukowsky_dp:.2f} bar")

In [ ]:
# Run simulation
print("Running water hammer simulation...")
wh_pipe.run()
print("Simulation complete!")

# Extract results
max_pressures = wh_pipe.getMaxPressureProfile()  # Pa
min_pressures = wh_pipe.getMinPressureProfile()  # Pa
final_pressures = wh_pipe.getPressureProfile()  # Pa
positions = np.linspace(0, wh_pipe.getLength(), len(max_pressures))

# Convert to bar
max_pressures_bar = [p / 1e5 for p in max_pressures]
min_pressures_bar = [p / 1e5 for p in min_pressures]
final_pressures_bar = [p / 1e5 for p in final_pressures]
initial_pressure = inlet.getPressure('bara')

print(f"\nResults:")
print(f"  Initial pressure: {initial_pressure:.2f} bar")
print(f"  Maximum pressure: {max(max_pressures_bar):.2f} bar")
print(f"  Minimum pressure: {min(min_pressures_bar):.2f} bar")
print(f"  Pressure rise: {max(max_pressures_bar) - initial_pressure:.2f} bar")
print(f"  Pressure drop: {initial_pressure - min(min_pressures_bar):.2f} bar")

In [ ]:
# Plot pressure envelope
fig, ax = plt.subplots(figsize=(14, 8))

# Plot pressure envelope
ax.fill_between(positions, min_pressures_bar, max_pressures_bar,
                alpha=0.3, color='red', label='Pressure envelope')
ax.plot(positions, max_pressures_bar, 'r-', linewidth=2, label='Maximum pressure')
ax.plot(positions, min_pressures_bar, 'b-', linewidth=2, label='Minimum pressure')
ax.axhline(y=initial_pressure, color='green', linestyle='--',
          linewidth=1.5, label='Initial pressure')

# Theoretical Joukowsky line
ax.axhline(y=initial_pressure + joukowsky_dp, color='orange',
          linestyle=':', linewidth=2, label='Joukowsky max')

ax.set_xlabel('Position along pipe (m)', fontsize=12)
ax.set_ylabel('Pressure (bar)', fontsize=12)
ax.set_title('Water Hammer - Pressure Envelope from Valve Closure', fontsize=14, fontweight='bold')
ax.legend(loc='best', fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Example 2: Time History at Specific Locations

Let's examine how pressure varies with time at different locations.

In [ ]:
# Get pressure history
pressure_history = wh_pipe.getPressureHistory()  # [time][location]
velocity_history = wh_pipe.getVelocityHistory()  # [time][location]

# Time array
time_steps = len(pressure_history)
time_array = np.linspace(0, wh_pipe.getSimulationTime(), time_steps)

# Select locations to plot: inlet, quarter, middle, three-quarters, outlet
n_sections = len(positions)
locations = [
    (0, 'Inlet (0 m)'),
    (n_sections // 4, f'Quarter ({positions[n_sections//4]:.0f} m)'),
    (n_sections // 2, f'Middle ({positions[n_sections//2]:.0f} m)'),
    (3 * n_sections // 4, f'Three-quarters ({positions[3*n_sections//4]:.0f} m)'),
    (n_sections - 1, f'Outlet ({positions[-1]:.0f} m)')
]

print(f"Time steps in simulation: {time_steps}")
print(f"Monitoring {len(locations)} locations")

In [ ]:
# Plot pressure vs time at different locations
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Pressure history
colors = ['blue', 'green', 'red', 'orange', 'purple']
for (idx, label), color in zip(locations, colors):
    pressure_time = [pressure_history[t][idx] / 1e5 for t in range(time_steps)]
    axes[0].plot(time_array, pressure_time, color=color, linewidth=1.5, label=label)

axes[0].axhline(y=initial_pressure, color='gray', linestyle='--', alpha=0.5, label='Initial')
axes[0].set_xlabel('Time (s)', fontsize=11)
axes[0].set_ylabel('Pressure (bar)', fontsize=11)
axes[0].set_title('Pressure Time History at Different Locations', fontsize=12, fontweight='bold')
axes[0].legend(loc='best', fontsize=9)
axes[0].grid(True, alpha=0.3)

# Velocity history
for (idx, label), color in zip(locations, colors):
    velocity_time = [velocity_history[t][idx] for t in range(time_steps)]
    axes[1].plot(time_array, velocity_time, color=color, linewidth=1.5, label=label)

axes[1].axhline(y=0, color='gray', linestyle='--', alpha=0.5)
axes[1].axvline(x=wh_pipe.getValveClosureTime(), color='red',
               linestyle=':', linewidth=2, alpha=0.7, label='Valve fully closed')
axes[1].set_xlabel('Time (s)', fontsize=11)
axes[1].set_ylabel('Velocity (m/s)', fontsize=11)
axes[1].set_title('Velocity Time History - Note Wave Reflections', fontsize=12, fontweight='bold')
axes[1].legend(loc='best', fontsize=9)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Example 3: Effect of Valve Closure Time

Slower valve closure reduces maximum pressure. Let's compare different closure times.

In [ ]:
# Test different valve closure times
closure_times = [0.05, 0.1, 0.5, 1.0, 2.0]  # seconds
max_pressure_rises = []

print("Simulating different valve closure times...\n")

for t_close in closure_times:
    # Create new pipe for each case
    test_pipe = pipeline.WaterHammerPipe(f"Pipe_t{t_close}", inlet.clone())
    test_pipe.setLength(1000.0)
    test_pipe.setDiameter(0.3)
    test_pipe.setRoughness(0.000046)
    test_pipe.setWallThickness(0.01)
    test_pipe.setNumberOfSections(50)
    test_pipe.setInitialVelocity(initial_velocity)
    test_pipe.setValveClosureTime(t_close)
    test_pipe.setSimulationTime(5.0)

    # Run simulation
    test_pipe.run()

    # Get maximum pressure
    max_p = max([p / 1e5 for p in test_pipe.getMaxPressureProfile()])
    pressure_rise = max_p - initial_pressure
    max_pressure_rises.append(pressure_rise)

    print(f"Closure time {t_close:.2f} s: Max pressure rise = {pressure_rise:.2f} bar")

print(f"\nTheoretical Joukowsky (instantaneous): {joukowsky_dp:.2f} bar")

In [ ]:
# Plot effect of closure time
fig, ax = plt.subplots(figsize=(12, 7))

ax.plot(closure_times, max_pressure_rises, 'o-', linewidth=2,
        markersize=10, color='darkred', label='Simulation results')
ax.axhline(y=joukowsky_dp, color='orange', linestyle='--',
          linewidth=2, label='Joukowsky (theoretical max)')

ax.set_xlabel('Valve Closure Time (s)', fontsize=12)
ax.set_ylabel('Maximum Pressure Rise (bar)', fontsize=12)
ax.set_title('Effect of Valve Closure Time on Water Hammer Pressure',
            fontsize=14, fontweight='bold')
ax.legend(loc='best', fontsize=11)
ax.grid(True, alpha=0.3)
ax.set_xlim(left=0)
ax.set_ylim(bottom=0)

# Add annotation
ax.text(0.6, 0.95, 'Slower closure reduces pressure surge',
       transform=ax.transAxes, fontsize=11, verticalalignment='top',
       bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.show()

## 6. Example 4: Cavitation Detection

If pressure drops below vapor pressure, cavitation occurs. This can happen during the negative pressure wave.

In [ ]:
# Create low-pressure scenario prone to cavitation
low_p_fluid = system.SystemSrkEos(298.15, 2.0)  # Only 2 bar initial pressure
low_p_fluid.addComponent("water", 1.0)
low_p_fluid.setMixingRule("classic")
low_p_fluid.init(0)

low_p_inlet = stream.Stream("low_p_inlet", low_p_fluid)
low_p_inlet.setFlowRate(50.0, "kg/sec")
low_p_inlet.setTemperature(298.15, "K")
low_p_inlet.setPressure(2.0, "bara")
low_p_inlet.run()

# Create pipe with fast valve closure
cav_pipe = pipeline.WaterHammerPipe("CavitationPipe", low_p_inlet)
cav_pipe.setLength(1000.0)
cav_pipe.setDiameter(0.3)
cav_pipe.setRoughness(0.000046)
cav_pipe.setWallThickness(0.01)
cav_pipe.setNumberOfSections(50)
cav_pipe.setInitialVelocity(2.5)  # Higher velocity
cav_pipe.setValveClosureTime(0.05)  # Very fast closure
cav_pipe.setSimulationTime(5.0)

print("Running cavitation-prone simulation...")
cav_pipe.run()
print("Simulation complete!")

# Get results
cav_max_p = [p / 1e5 for p in cav_pipe.getMaxPressureProfile()]
cav_min_p = [p / 1e5 for p in cav_pipe.getMinPressureProfile()]
cav_positions = np.linspace(0, cav_pipe.getLength(), len(cav_max_p))

# Check for cavitation (pressure below vapor pressure ~0.03 bar at 25°C)
vapor_pressure = 0.03  # bar
cavitation_detected = any(p < vapor_pressure for p in cav_min_p)

print(f"\nResults:")
print(f"  Initial pressure: {low_p_inlet.getPressure('bara'):.2f} bar")
print(f"  Maximum pressure: {max(cav_max_p):.2f} bar")
print(f"  Minimum pressure: {min(cav_min_p):.4f} bar")
print(f"  Vapor pressure (25°C): {vapor_pressure:.4f} bar")
print(f"  Cavitation detected: {cavitation_detected}")

In [ ]:
# Plot with cavitation zone highlighted
fig, ax = plt.subplots(figsize=(14, 8))

ax.fill_between(cav_positions, cav_min_p, cav_max_p,
                alpha=0.3, color='red', label='Pressure envelope')
ax.plot(cav_positions, cav_max_p, 'r-', linewidth=2, label='Maximum pressure')
ax.plot(cav_positions, cav_min_p, 'b-', linewidth=2, label='Minimum pressure')
ax.axhline(y=vapor_pressure, color='purple', linestyle='--',
          linewidth=2, label=f'Vapor pressure ({vapor_pressure} bar)')

# Highlight cavitation zone
if cavitation_detected:
    ax.fill_between(cav_positions, 0, vapor_pressure,
                    where=[p < vapor_pressure for p in cav_min_p],
                    alpha=0.4, color='purple', label='Cavitation zone')

ax.set_xlabel('Position along pipe (m)', fontsize=12)
ax.set_ylabel('Pressure (bar)', fontsize=12)
ax.set_title('Water Hammer with Cavitation Detection', fontsize=14, fontweight='bold')
ax.legend(loc='best', fontsize=10)
ax.grid(True, alpha=0.3)
ax.set_ylim(bottom=0)

plt.tight_layout()
plt.show()

## 7. Example 5: Inclined Pipeline

Elevation changes affect water hammer due to hydrostatic pressure differences.

In [ ]:
# Create uphill pipeline
uphill_fluid = system.SystemSrkEos(298.15, 15.0)  # 15 bar to handle elevation
uphill_fluid.addComponent("water", 1.0)
uphill_fluid.setMixingRule("classic")
uphill_fluid.init(0)

uphill_inlet = stream.Stream("uphill_inlet", uphill_fluid)
uphill_inlet.setFlowRate(60.0, "kg/sec")
uphill_inlet.setTemperature(298.15, "K")
uphill_inlet.setPressure(15.0, "bara")
uphill_inlet.run()

# Create inclined pipe
inclined_pipe = pipeline.WaterHammerPipe("InlinedPipe", uphill_inlet)
inclined_pipe.setLength(1000.0)
inclined_pipe.setDiameter(0.3)
inclined_pipe.setRoughness(0.000046)
inclined_pipe.setWallThickness(0.01)
inclined_pipe.setNumberOfSections(50)
inclined_pipe.setInitialVelocity(2.0)

# Set elevation profile (uphill 50 m over 1000 m = 2.86 degrees)
elevations = np.linspace(0, 50, 51)  # 0 to 50 m elevation gain
inclined_pipe.setElevationProfile(elevations.tolist())

inclined_pipe.setValveClosureTime(0.1)
inclined_pipe.setSimulationTime(5.0)

print("Running inclined pipe simulation...")
inclined_pipe.run()
print("Simulation complete!")

# Get results
inc_max_p = [p / 1e5 for p in inclined_pipe.getMaxPressureProfile()]
inc_min_p = [p / 1e5 for p in inclined_pipe.getMinPressureProfile()]
inc_positions = np.linspace(0, inclined_pipe.getLength(), len(inc_max_p))

print(f"\nInclined pipe results:")
print(f"  Elevation change: {elevations[-1]:.0f} m")
print(f"  Inlet max pressure: {inc_max_p[0]:.2f} bar")
print(f"  Outlet max pressure: {inc_max_p[-1]:.2f} bar")
print(f"  Pressure difference due to elevation: {elevations[-1] * 0.981:.2f} bar (approx)")

In [ ]:
# Plot inclined pipe results
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Elevation profile
axes[0].fill_between(inc_positions, elevations[:-1], 0, alpha=0.3, color='brown')
axes[0].plot(inc_positions, elevations[:-1], 'k-', linewidth=2)
axes[0].set_ylabel('Elevation (m)', fontsize=11)
axes[0].set_title('Pipeline Elevation Profile', fontsize=12, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Pressure envelope
axes[1].fill_between(inc_positions, inc_min_p, inc_max_p,
                     alpha=0.3, color='red', label='Pressure envelope')
axes[1].plot(inc_positions, inc_max_p, 'r-', linewidth=2, label='Maximum pressure')
axes[1].plot(inc_positions, inc_min_p, 'b-', linewidth=2, label='Minimum pressure')
axes[1].set_xlabel('Position along pipe (m)', fontsize=11)
axes[1].set_ylabel('Pressure (bar)', fontsize=11)
axes[1].set_title('Pressure Envelope - Note Elevation Effect', fontsize=12, fontweight='bold')
axes[1].legend(loc='best', fontsize=10)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 8. Wave Speed Calculation

Let's examine how pipe properties affect wave speed.

In [ ]:
# Calculate wave speed for different pipe materials
def calculate_wave_speed(K, rho, D, E, t):
    """
    Calculate wave speed in pipe
    K: Bulk modulus of fluid (Pa)
    rho: Density (kg/m³)
    D: Pipe diameter (m)
    E: Young's modulus of pipe (Pa)
    t: Wall thickness (m)
    """
    return np.sqrt((K / rho) / (1 + K * D / (E * t)))

# Water properties
K_water = 2.2e9  # Pa
rho_water = 1000  # kg/m³

# Pipe properties
D = 0.3  # m
t = 0.01  # m

# Different materials
materials = {
    'Steel': 200e9,
    'Copper': 120e9,
    'PVC': 3.5e9,
    'HDPE': 1.0e9,
    'Concrete': 30e9
}

print("Wave speeds in different pipe materials:\n")
print(f"{'Material':<15} {'Youngs Modulus (GPa)':<25} {'Wave Speed (m/s)':<20}")
print("-" * 60)

wave_speeds = {}
for material, E in materials.items():
    a = calculate_wave_speed(K_water, rho_water, D, E, t)
    wave_speeds[material] = a
    print(f"{material:<15} {E/1e9:<25.1f} {a:<20.1f}")

print(f"\nNote: Higher stiffness (E) → Higher wave speed → Higher pressure surge")

In [ ]:
# Plot wave speeds
fig, ax = plt.subplots(figsize=(12, 7))

materials_list = list(wave_speeds.keys())
speeds = list(wave_speeds.values())

bars = ax.barh(materials_list, speeds, color=['steelblue', 'coral', 'lightgreen', 'gold', 'lightgray'])
ax.set_xlabel('Wave Speed (m/s)', fontsize=12)
ax.set_ylabel('Pipe Material', fontsize=12)
ax.set_title('Pressure Wave Speed in Different Pipe Materials', fontsize=14, fontweight='bold')
ax.grid(True, axis='x', alpha=0.3)

# Add value labels
for i, (material, speed) in enumerate(wave_speeds.items()):
    ax.text(speed + 20, i, f'{speed:.0f} m/s', va='center', fontsize=10)

plt.tight_layout()
plt.show()

## 9. Practical Design Guidelines

### Preventing Water Hammer Damage

1. **Slow Valve Closure:**
   - Critical time: $t_c = 2L/a$ (round-trip time)
   - Closure time > $t_c$ significantly reduces surge

2. **Surge Protection Devices:**
   - Air chambers (accumulators)
   - Surge relief valves
   - One-way surge tanks
   - Flywheel on pumps

3. **Maximum Allowable Pressure:**
   - Typically 1.5× design pressure
   - Check pipe class rating
   - Consider fatigue from repeated events

4. **Cavitation Prevention:**
   - Maintain pressure above vapor pressure
   - Avoid low points with insufficient pressure
   - Air release valves at high points

In [ ]:
# Calculate critical closure time for our example
L = 1000.0  # m
a = 1200.0  # m/s (typical for water in steel)

t_critical = 2 * L / a

print(f"Critical Closure Time Calculation:")
print(f"  Pipeline length: {L:.0f} m")
print(f"  Wave speed: {a:.0f} m/s")
print(f"  Critical time (2L/a): {t_critical:.3f} s")
print(f"\nRecommendation:")
print(f"  Valve closure time should be > {t_critical:.2f} s to minimize surge")
print(f"  Recommended closure time: > {t_critical * 2:.1f} s for safety margin")

## 10. Summary of Capabilities

### What Can Be Calculated:

1. **Pressure Analysis:**
   - Maximum and minimum pressures along pipeline
   - Pressure time history at any location
   - Pressure envelope (upper and lower bounds)
   - Joukowsky pressure rise validation

2. **Velocity Analysis:**
   - Velocity changes during transient
   - Flow reversal detection
   - Time to zero velocity

3. **Wave Propagation:**
   - Wave speed calculation
   - Reflection at boundaries
   - Wave period and frequency

4. **Cavitation:**
   - Cavitation zone identification
   - Vapor pressure comparison
   - Column separation locations

5. **Design Parameters:**
   - Effect of valve closure time
   - Effect of pipe material
   - Effect of pipe diameter and length
   - Elevation and inclination effects

### Recommended Applications:

- **Pipeline design** - sizing and pressure rating
- **Valve selection** - closure time requirements
- **Pump station design** - startup/shutdown procedures
- **Safety analysis** - maximum pressure assessment
- **Emergency shutdown** systems design
- **Surge protection** device sizing
- **Operator training** - understanding transient behavior

### Key References:

1. Wylie, E.B. and Streeter, V.L. (1993) - *Fluid Transients in Systems*
2. Chaudhry, M.H. (2014) - *Applied Hydraulic Transients*
3. Thorley, A.R.D. (2004) - *Fluid Transients in Pipeline Systems*
4. Joukowsky, N. (1900) - Original water hammer theory